In [2]:
import pandas as pd
from pathlib import Path
import yahoo_fin.stock_info as si
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Read in News Dataset

In [142]:
df = pd.read_csv('article_data/us_equities_news_dataset.csv')
df.set_index(df.release_date, inplace=True)

In [144]:
df.drop(columns = ['release_date'], axis=1, inplace=True)

In [145]:
df.sort_index(inplace=True)
df.head()

,id,ticker,title,category,content,provider,url,article_id
release_date,,,,,,,,
2008-10-02,227750,KYOCY,Nikkei down 1 4 pct on economy fears autos drop,news,Nikkei down 1 4 pct hits lowest point in ove...,Reuters,https://www.investing.com/news/forex-news/nikk...,669
2008-10-03,327113,TGT,FOREX Dollar poised for biggest weekly gain in...,news,Money market squeeze ECB shift fuel dollar ...,Reuters,https://www.investing.com/news/forex-news/fore...,671
2008-10-03,435744,C,GLOBAL MARKETS Stocks rally on Wells Wachovia ...,news,U S stocks rally dollar gains on European w...,Reuters,https://www.investing.com/news/forex-news/glob...,682
2008-10-03,441802,WFC,GLOBAL MARKETS Stocks hold up after Wells Wach...,news,Europe stocks U S stock futures up on Well...,Reuters,https://www.investing.com/news/forex-news/glob...,673
2008-10-03,441803,WFC,GLOBAL MARKETS U S bailout plan passes inves...,news,U S House of Representatives approves bailou...,Reuters,https://www.investing.com/news/forex-news/glob...,686


In [146]:
# Create Copy of dataframe to manipulate
df2 = df.copy()
df2 = df2.loc['2016-01-01':]

In [149]:
df_ticker_list = df2['ticker'].unique()
sp500_tickers = si.tickers_sp500()

In [150]:
# Gets list of S&P 500 tickers found in news dataset
df_sp500_list = [x for x in sp500_tickers if x in df_ticker_list]

In [151]:
len(df_sp500_list)

340

In [152]:
# Gets list of S&P 500 tickers NOT found in news dataset
missing_tickers = [x for x in sp500_tickers if x not in df_sp500_list]

In [153]:
len(missing_tickers)

164

In [154]:
# Add Tesla to list of tickers
df_sp500_list.append('TSLA')

In [155]:
len(df_sp500_list)

341

In [156]:
# Set Dataframe such that only rows containing S&P 500 tickers and Tesla are found
df2 = df2.loc[df2['ticker'].isin(df_sp500_list)]

In [158]:
# Create a new dataframe with just the headlines and ticker
news_df = df2[['ticker', 'title']]

In [159]:
# Create vader Sentiment analyzer
vader = SentimentIntensityAnalyzer()
# Lambda function to run vader sentiment on title column
f = lambda title: vader.polarity_scores(title)['compound']
# Apply sentiment to title column and put values in a new column named title_sentiment
news_df['title_sentiment'] = news_df.title.apply(f)

/var/folders/g0/x29rhtxs77n94c0nv9tn68km0000gn/T/ipykernel_59536/3955821164.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [162]:
# Sort dataframe rows by ticker and release date
news_df.sort_values(by=['ticker', 'release_date'], ascending=True, inplace=True)

/var/folders/g0/x29rhtxs77n94c0nv9tn68km0000gn/T/ipykernel_59536/629580004.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [1]:
news_df.tail()

NameError: name 'news_df' is not defined

In [166]:
news_df.describe()

,title_sentiment
count,144657.000000
mean,0.066251
std,0.310186
min,-0.942200
25%,0.000000
50%,0.000000
75%,0.273200
max,0.933700


In [164]:
# Create csv out of news_df table
filepath = Path('article_data/news_sentiment.csv')
news_df.to_csv(filepath)